In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

/home/ic-lab/miniconda3/envs/torch_v1.9_gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task_datasets = []

# Task 1
task1_data = np.load('./cwru1_X_train.npy')
task1_labels = np.load('./cwru2_y_train.npy')

# Task 2
task2_data = np.load('./cwru2_X_train.npy')
task2_labels = np.load('./cwru2_y_train.npy')

# Task 3
task3_data = np.load('./cwru3_X_train.npy')
task3_labels = np.load('./cwru3_y_train.npy')

# Task 4
task4_data = np.load('./cwru4_X_train.npy')
task4_labels = np.load('./cwru4_y_train.npy')

In [3]:
# Convert the data to PyTorch tensors
# Task 1
task1_data = torch.from_numpy(task1_data).transpose(1, 2)
task1_labels = torch.from_numpy(task1_labels)

# Task 2
task2_data = torch.from_numpy(task2_data).transpose(1, 2)
task2_labels = torch.from_numpy(task2_labels)

# Task 3
task3_data = torch.from_numpy(task3_data).transpose(1, 2)
task3_labels = torch.from_numpy(task3_labels)

# Task 4
task4_data = torch.from_numpy(task4_data).transpose(1, 2)
task4_labels = torch.from_numpy(task4_labels)

In [4]:
# dataset for task 1~4
dataset1 = torch.utils.data.TensorDataset(task1_data, task1_labels)
dataset2 = torch.utils.data.TensorDataset(task2_data, task2_labels)
dataset3 = torch.utils.data.TensorDataset(task3_data, task3_labels)
dataset4 = torch.utils.data.TensorDataset(task4_data, task4_labels)

In [5]:
# 划分训练集和测试集
train_size = int(0.8 * len(dataset1))
test_size = len(dataset1) - train_size

train_dataset1, test_dataset1 = random_split(dataset1, [train_size, test_size])
train_dataset2, test_dataset2 = random_split(dataset2, [train_size, test_size])
train_dataset3, test_dataset3 = random_split(dataset3, [train_size, test_size])
train_dataset4, test_dataset4 = random_split(dataset4, [train_size, test_size])

In [6]:
task_datasets.append((train_dataset1, test_dataset1))
task_datasets.append((train_dataset2, test_dataset2))
task_datasets.append((train_dataset3, test_dataset3))
task_datasets.append((train_dataset4, test_dataset4))

In [7]:
# 定义WDCNN模型
class WDCNN(nn.Module):
    def __init__(self, num_classes):
        super(WDCNN, self).__init__()
        self.conv1 = nn.Conv1d(2, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 256, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [8]:
num_classes = 10

# Define the meta-learning algorithm (MAML)
def maml_bearing_fault_diagnosis(tasks, num_epochs_inner=5, num_epochs_outer=10, batch_size=8, lr_inner=0.01, lr_outer=0.001):
    # Initialize the WDCNN backbone
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = WDCNN(num_classes).to(device)
    model.double()
    model.train()
    
    # Define the optimizer for inner-loop adaptation
    optimizer_inner = optim.SGD(model.parameters(), lr=lr_inner)
    
    # Define the optimizer for outer-loop updates
    optimizer_outer = optim.Adam(model.parameters(), lr=lr_outer)
    
    # Define the optimizer for the meta-updates step
    meta_optimizer = optim.Adam(model.parameters(), lr=lr_outer)
    
    # Meta-training loop
    for epoch_outer in range(num_epochs_outer):
        # Accumulate the meta-gradients
        meta_gradients = [torch.zeros_like(param) for param in model.parameters()]
        
        for task in tasks:
            # Split the data into support set and query set
            support_set, query_set = task[1], task[0]
            
            # Inner-loop adaptation
            for epoch_inner in range(num_epochs_inner):
                for inputs, labels in DataLoader(support_set, batch_size=batch_size, shuffle=True):
                    inputs = inputs.to(device, dtype=torch.double)
                    labels = labels.to(device, dtype=torch.long)
                    
                    optimizer_inner.zero_grad()
                    
                    outputs = model(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, labels)
                    loss.backward()
                    optimizer_inner.step()
            
            # Compute the meta-gradients
            for inputs, labels in DataLoader(query_set, batch_size=batch_size, shuffle=True):
                inputs = inputs.to(device, dtype=torch.double)
                labels = labels.to(device, dtype=torch.long)
                
                outputs = model(inputs)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                
                # MAML
                # gradients = torch.autograd.grad(loss, model.parameters(), create_graph=True)
                # meta_gradients = [meta_gradients[i] + gradients[i] for i in range(len(gradients))]
                
                # FOMAML
                # Perform the meta-update step
                meta_optimizer.zero_grad()
                loss.backward()
                meta_optimizer.step()
                
        # MAML
        # Update the model parameters with meta-gradients
        # for param, meta_gradient in zip(model.parameters(), meta_gradients):
        #     param.grad = meta_gradient / len(tasks)
        # optimizer_outer.step()
    
    # Return the trained model
    return model


In [10]:
# Define the tasks (training and testing sets)
# tasks = task_datasets
# Train the meta-learning model
meta_model = maml_bearing_fault_diagnosis(task_datasets)
# save model
torch.save(meta_model.state_dict(), './parameters/meta_model.pt')

In [19]:
model = WDCNN(num_classes).to(device)
model.load_state_dict(torch.load('./parameters/meta_model.pt'))
model.double()

WDCNN(
  (conv1): Conv1d(2, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
# Evaluate the model on testing sets of all tasks
for i, task in enumerate(task_datasets):
    # test_set = task[1]
    test_loader = torch.utils.data.DataLoader(task[1], batch_size=32, shuffle=False)
    
    model.eval()
    correct = 0.0
    total = 0.0
    task_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device, dtype=torch.double)
            labels = labels.to(device, dtype=torch.long)

            # inputs = inputs.to(device)  # 移动到设备上
            # labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            task_loss += loss.item()

    accuracy = 100 * correct / total
    loss_avg = task_loss / len(test_loader)
    # task_accuracies.append(accuracy)
    # task_losses.append(loss_avg)

    print(f"Task {i+1} Accuracy: {accuracy:.2f}%")
    print(f"Task {i+1} Loss: {loss_avg:.4f}")

Task 1 Accuracy: 82.54%
Task 1 Loss: 1.0807
Task 2 Accuracy: 80.64%
Task 2 Loss: 1.0018
Task 3 Accuracy: 88.89%
Task 3 Loss: 0.4365
Task 4 Accuracy: 99.96%
Task 4 Loss: 0.0007
